# ES → BBF Salesforce BAN Migration

This notebook migrates Billing_Invoice__c records from ES Salesforce to BAN__c in BBF Salesforce.

## Prerequisites
- **Account migration must be completed first**
- ES Accounts with `BBF_New_Id__c` populated = parent Account already exists in BBF
- ES Billing_Invoice__c records marked with `BBF_Ban__c = True` = records to migrate

## Object Mapping
- **ES Source:** Billing_Invoice__c (45 fields)
- **BBF Target:** BAN__c (91 fields)

## Process Overview
1. Connect to both ES (source) and BBF (target) Salesforce orgs
2. Query Billing_Invoice__c from ES where:
   - `BBF_Ban__c = True` (marked for migration)
   - Parent Account has `BBF_New_Id__c` populated (Account already migrated)
   - `BBF_New_Id__c` is empty (not yet migrated)
3. Transform ES Billing_Invoice__c for BBF BAN__c schema:
   - Map `Account__c` → ES Account's `BBF_New_Id__c` (BBF Account ID)
   - Map billing fields
   - Set required `busUnit__c` picklist to 'EVS'
   - Add `ES_Legacy_ID__c` = ES Billing_Invoice.Id (for tracking)
4. Insert BAN__c to BBF Salesforce
5. Update ES Billing_Invoice__c with `BBF_New_Id__c` = BBF BAN.Id
6. Create ID mapping: ES Billing_Invoice ID → BBF BAN ID
7. Output results to Excel with color-coded status

## Field Tracking Strategy
**In BBF BAN__c:** `ES_Legacy_ID__c` stores original ES Billing_Invoice ID
- Text(18), External ID, Unique

**In ES Billing_Invoice__c:**
- `BBF_Ban__c` = True (checkbox) - **pre-set to identify records to migrate**
- `BBF_New_Id__c` stores new BBF BAN ID after migration (Text 18)

## Safety
- `TEST_MODE = True` by default (limits to 10 BANs)
- Only migrates Billing_Invoice__c where `BBF_Ban__c = True`
- Skips Billing_Invoice__c where parent Account not yet migrated
- Skips Billing_Invoice__c already migrated (`BBF_New_Id__c` populated)

In [11]:
# === SETUP & IMPORTS ===

import sys
import pandas as pd
from simple_salesforce import Salesforce
from openpyxl import Workbook
from openpyxl.styles import Font, PatternFill, Alignment, Border, Side
from datetime import datetime
import os

print(f"Python: {sys.executable}")
print(f"Pandas: {pd.__version__}")
print("✅ Set-up successful")

Python: C:\Users\vjero\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe
Pandas: 2.2.3
✅ Set-up successful


In [12]:
# === CONFIGURATION ===

# ES UAT Credentials
ES_USERNAME = "sfdcapi@everstream.net.uat"
ES_PASSWORD = "ZXasqw1234!@#$"
ES_TOKEN = "X0ation2CNmK5C0pV94M6vFYS"
ES_DOMAIN = "test"

# # ES (Source) Credentials - Production
# ES_USERNAME = "sfdcapi@everstream.net"
# ES_PASSWORD = "pV4CAxns8DQtJsBq!"
# ES_TOKEN = "r1uoYiusK19RbrflARydi86TA"
# ES_DOMAIN = "login"  # or 'test' for sandbox

# BBF (Target) Credentials
BBF_USERNAME = "vlettau@everstream.net"
BBF_PASSWORD = "MNlkpo0987)(*&"
BBF_TOKEN = "I4xmQLmm03cXl1O9qI2Z3XAAX"
BBF_DOMAIN = "test"  # or 'login' for production

# Migration Options
TEST_MODE = False  # ⚠️ Set to False to migrate ALL BANs
TEST_LIMIT = 10  # Only used when TEST_MODE = True

# 👤 BAN Owner - Set all migrated BANs to this user
OWNER_ID = "005Ea00000ZOGFZIA5"  # Same as Account/Contact migration

# 🏢 Default Business Unit - Required picklist field in BBF
# ES records being migrated should be assigned to EVS business unit
DEFAULT_BUS_UNIT = "EVS"  # EverStream business unit in BBF

# Output Configuration
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
output_file = f"es_bbf_ban_migration_{timestamp}.xlsx"

print("📋 Configuration loaded")
print(f"   TEST_MODE: {TEST_MODE}")
print(f"   Owner ID: {OWNER_ID}")
print(f"   Default Business Unit: {DEFAULT_BUS_UNIT}")
print(f"   Output: {output_file}")
print("\n⚠️  Note: Bulk API automatically handles batching (200 records/batch)")

📋 Configuration loaded
   TEST_MODE: False
   Owner ID: 005Ea00000ZOGFZIA5
   Default Business Unit: EVS
   Output: es_bbf_ban_migration_20260123_120158.xlsx

⚠️  Note: Bulk API automatically handles batching (200 records/batch)


In [13]:
# === CONNECT TO SALESFORCE ORGS ===

print("=" * 80)
print("CONNECTING TO SALESFORCE ORGS")
print("=" * 80)

# Connect to ES (source)
print("\n📌 Connecting to ES (source)...")
es_sf = Salesforce(
    username=ES_USERNAME,
    password=ES_PASSWORD,
    security_token=ES_TOKEN,
    domain=ES_DOMAIN,
)
print(f"✅ Connected to ES: {es_sf.sf_instance}")

# Connect to BBF (target)
print("\n📌 Connecting to BBF (target)...")
bbf_sf = Salesforce(
    username=BBF_USERNAME,
    password=BBF_PASSWORD,
    security_token=BBF_TOKEN,
    domain=BBF_DOMAIN,
)
print(f"✅ Connected to BBF: {bbf_sf.sf_instance}")

CONNECTING TO SALESFORCE ORGS

📌 Connecting to ES (source)...
✅ Connected to ES: everstream--uat.sandbox.my.salesforce.com

📌 Connecting to BBF (target)...
✅ Connected to BBF: bluebirdnetwork--full.sandbox.my.salesforce.com


In [ ]:
# === QUERY ES BILLING_INVOICE__c (BANs) ===
# Only Billing_Invoice__c where:
# 1. Parent Account has BBF_New_Id__c populated (Account already migrated)
# 2. Billing_Invoice__c does NOT have BBF_New_Id__c populated (not yet migrated)

print("\n" + "=" * 80)
print("QUERYING ES BILLING_INVOICE__c (BANs)")
print("=" * 80)

# Build query - filter for BBF_Ban__c = True (records marked for migration)
# and parent Account has BBF_New_Id__c populated (Account already migrated)
query = """
    SELECT Id, Name, Account__c, Account__r.BBF_New_Id__c, Account__r.Name,
           Account_Number__c, Account_Name__c,
           Billing_Address_1__c, Billing_Address_2__c,
           Billing_City__c, Billing_State__c, Billing_ZIP__c,
           Billing_E_mail__c, Additional_Emails__c,
           Payment_Terms__c, Invoice_Delivery_Preference__c, Invoice_cycle_cd__c,
           Disable_Late_Fees__c, Late_Fee_Percentage__c,
           Suppress_Invoice_Generation__c, Suppress_Past_Due_Notifications__c,
           Address_Verified__c, Address_Verified_On__c, AddressReturnCode__c,
           Disabled__c, Description__c, Billing_Notes__c,
           Automatic_Bill_Payment_Authorized__c,
           Detailed_Tax_Breakout__c, Sent_to_Third_party__c,
           AP_Contact__c, BBF_Ban__c
    FROM Billing_Invoice__c
    WHERE BBF_Ban__c = true
      AND Account__r.BBF_New_Id__c != null 
      AND Account__r.BBF_New_Id__c != ''
      AND (BBF_New_Id__c = null OR BBF_New_Id__c = '')
"""

# Add limit for test mode
if TEST_MODE:
    query += f" LIMIT {TEST_LIMIT}"

print(f"Query: {query}")
print("\nExecuting query...")

result = es_sf.query_all(query)
es_bans_raw = result["records"]

print(f"✅ Found {len(es_bans_raw)} Billing_Invoice__c records from ES query")

# =============================================================================
# DUPLICATE PREVENTION: Check BBF for already-migrated records
# If record exists in BBF, get its Id to update ES BBF_New_Id__c
# =============================================================================
print("\n" + "-" * 80)
print("DUPLICATE PREVENTION: Checking BBF for existing ES_Legacy_ID__c...")
print("-" * 80)

bbf_existing_query = """
SELECT Id, ES_Legacy_ID__c 
FROM BAN__c 
WHERE ES_Legacy_ID__c != null
"""
bbf_existing_result = bbf_sf.query_all(bbf_existing_query)

# Build lookup: ES_Legacy_ID__c -> BBF Id
existing_bbf_lookup = {r['ES_Legacy_ID__c']: r['Id'] for r in bbf_existing_result['records']}

print(f"   Found {len(existing_bbf_lookup)} BAN__c records already in BBF")

# Separate: records to migrate vs records that need ES BBF_New_Id__c sync
es_bans = []
es_needs_sync = []  # Records that exist in BBF but ES.BBF_New_Id__c is null

for ban in es_bans_raw:
    if ban['Id'] in existing_bbf_lookup:
        # Already in BBF - need to sync ES.BBF_New_Id__c
        es_needs_sync.append({
            'es_id': ban['Id'],
            'bbf_id': existing_bbf_lookup[ban['Id']]
        })
    else:
        # Not in BBF - need to migrate
        es_bans.append(ban)

print(f"   Records to migrate (not in BBF): {len(es_bans)}")
print(f"   Records to sync (in BBF, ES.BBF_New_Id__c missing): {len(es_needs_sync)}")

# Sync ES.BBF_New_Id__c for records that already exist in BBF
if len(es_needs_sync) > 0:
    print(f"\n📌 Syncing {len(es_needs_sync)} ES Billing_Invoice__c BBF_New_Id__c values...")
    
    sync_updates = [{'Id': item['es_id'], 'BBF_New_Id__c': item['bbf_id']} for item in es_needs_sync]
    
    try:
        sync_results = es_sf.bulk.Billing_Invoice__c.update(sync_updates)
        sync_success = sum(1 for r in sync_results if r['success'])
        sync_failed = sum(1 for r in sync_results if not r['success'])
        
        print(f"   ✅ Synced: {sync_success}")
        print(f"   ❌ Failed to sync: {sync_failed}")
        
        if sync_failed > 0:
            print("   First 5 sync failures:")
            fail_count = 0
            for i, r in enumerate(sync_results):
                if not r['success'] and fail_count < 5:
                    print(f"     - {sync_updates[i]['Id']}: {r['errors']}")
                    fail_count += 1
    except Exception as e:
        print(f"   ❌ Error syncing: {e}")

print(f"\n✅ {len(es_bans)} Billing_Invoice__c records to migrate (after duplicate check)")

if len(es_bans) > 0:
    sample = es_bans[0]
    print(f"\nSample Billing_Invoice__c:")
    print(f"  ID:          {sample['Id']}")
    print(f"  Name:        {sample.get('Name', 'N/A')}")
    print(f"  ES Account:  {sample.get('Account__c', 'N/A')}")
    print(
        f"  BBF Account (target): {sample.get('Account__r', {}).get('BBF_New_Id__c', 'N/A')}"
    )
elif TEST_MODE:
    print("\n⚠️  No unmigrated Billing_Invoice__c found in test set")
    print("   Check: Are there ES Accounts with BBF_New_Id__c populated?")
else:
    print(
        "\n✅ All Billing_Invoice__c records have been migrated (or no eligible records found)!"
    )

In [15]:
# === TRANSFORM FOR BBF BAN__c ===

print("\n" + "=" * 80)
print("TRANSFORMING BILLING_INVOICE__c FOR BBF BAN__c")
print("=" * 80)

# Payment Terms Translation: ES → BBF
# ES uses no space (NET30), BBF uses space (NET 30)
PAYMENT_TERMS_MAP = {
    "NET30": "NET 30",
    "NET45": "NET 45",
    "NET60": "NET 60",
    "NET30,NET45,NET60 (default)": "NET 30",  # Map default to NET 30
    "NET 30": "NET 30",  # Already correct format
    "NET 45": "NET 45",  # Already correct format
    "NET 60": "NET 60",  # Already correct format
    "Due On Receipt": "Due On Receipt",  # Direct match
}

# Valid BBF Payment Terms values
VALID_BBF_PAYMENT_TERMS = {"NET 30", "NET 45", "NET 60", "Due On Receipt"}


def translate_payment_terms(es_value):
    """Translate ES Payment_Terms__c to BBF format. Returns (translated_value, error_msg)"""
    if es_value is None or es_value == "":
        return None, None  # Null is OK, field is nillable

    # Check direct mapping
    if es_value in PAYMENT_TERMS_MAP:
        return PAYMENT_TERMS_MAP[es_value], None

    # Check if already valid BBF value
    if es_value in VALID_BBF_PAYMENT_TERMS:
        return es_value, None

    # No valid translation found
    return None, f"Invalid Payment_Terms__c value: '{es_value}' - no BBF equivalent"


print("Payment Terms Mapping:")
for es_val, bbf_val in PAYMENT_TERMS_MAP.items():
    print(f"   '{es_val}' → '{bbf_val}'")

bbf_bans = []
skipped_no_bbf_account = []
skipped_invalid_payment_terms = []

for es_ban in es_bans:
    # Get the BBF Account ID from the parent Account
    bbf_account_id = None
    if es_ban.get("Account__r") and es_ban["Account__r"].get("BBF_New_Id__c"):
        bbf_account_id = es_ban["Account__r"]["BBF_New_Id__c"]

    # Safety check: Skip if no BBF Account ID (shouldn't happen due to query filter)
    if not bbf_account_id:
        skipped_no_bbf_account.append(
            {
                "es_id": es_ban["Id"],
                "name": es_ban.get("Name", "N/A"),
                "es_account_id": es_ban.get("Account__c"),
                "reason": "No BBF Account ID found",
            }
        )
        continue

    # Translate Payment Terms
    es_payment_terms = es_ban.get("Payment_Terms__c")
    bbf_payment_terms, payment_terms_error = translate_payment_terms(es_payment_terms)

    if payment_terms_error:
        skipped_invalid_payment_terms.append(
            {
                "es_id": es_ban["Id"],
                "name": es_ban.get("Name", "N/A"),
                "es_account_id": es_ban.get("Account__c"),
                "bbf_account_id": bbf_account_id,
                "reason": payment_terms_error,
                "es_value": es_payment_terms,
            }
        )
        continue

    # Build BBF BAN__c record
    # Combine Billing_Address_1 and Billing_Address_2 for BBF Billing_Street__c (max 100 chars)
    billing_street = ""
    if es_ban.get("Billing_Address_1__c"):
        billing_street = es_ban["Billing_Address_1__c"]
    if es_ban.get("Billing_Address_2__c"):
        billing_street += " " + es_ban["Billing_Address_2__c"]
    billing_street = billing_street.strip()[:100]  # Truncate to BBF field length

    bbf_name = "EV-" + es_ban.get("Name")

    bbf_ban = {
        # 🔗 CRITICAL: Required lookup to BBF Account
        "Account__c": bbf_account_id,
        # 🏢 CRITICAL: Required picklist - Business Unit
        "busUnit__c": DEFAULT_BUS_UNIT,
        # Name field - BAN Name/Number
        "Name": bbf_name,
        # Billing Address fields
        "Billing_Street__c": billing_street if billing_street else None,
        "Billing_City__c": es_ban.get("Billing_City__c"),
        "Billing_State__c": es_ban.get("Billing_State__c"),
        "Billing_PostalCode__c": es_ban.get("Billing_ZIP__c"),
        # Billing Company Name (from ES Account Name)
        "Billing_Company_Name__c": es_ban.get("Account_Name__c"),
        # BAN Description
        "BAN_Description__c": es_ban.get("Description__c"),
        # Payment Terms - translated from ES to BBF format
        "Payment_Terms__c": bbf_payment_terms,
        # General Notes
        "General_Description__c": es_ban.get("Billing_Notes__c"),
        # 👤 Set owner
        # "OwnerId": OWNER_ID,
        # 🔗 Store ES Billing_Invoice ID for tracking
        "ES_Legacy_ID__c": es_ban["Id"],
    }

    bbf_bans.append(bbf_ban)

print(f"✅ Transformed {len(bbf_bans)} BANs")
if len(bbf_bans) > 0:
    print(
        f"   - Mapped {len([k for k in bbf_bans[0].keys() if bbf_bans[0][k] is not None])} fields per BAN"
    )
    print(f"   - Set OwnerId to {OWNER_ID}")
    print(f"   - Set busUnit__c to {DEFAULT_BUS_UNIT}")
    print(f"   - Set ES_Legacy_ID__c for tracking")

if len(skipped_no_bbf_account) > 0:
    print(f"\n⚠️  Skipped {len(skipped_no_bbf_account)} BANs (no BBF Account ID found)")
    for skip in skipped_no_bbf_account[:5]:
        print(f"   - {skip['name']} (ES Account: {skip['es_account_id']})")

if len(skipped_invalid_payment_terms) > 0:
    print(
        f"\n⚠️  Skipped {len(skipped_invalid_payment_terms)} BANs (invalid Payment_Terms__c)"
    )
    for skip in skipped_invalid_payment_terms[:5]:
        print(f"   - {skip['name']}: {skip['reason']}")


TRANSFORMING BILLING_INVOICE__c FOR BBF BAN__c
Payment Terms Mapping:
   'NET30' → 'NET 30'
   'NET45' → 'NET 45'
   'NET60' → 'NET 60'
   'NET30,NET45,NET60 (default)' → 'NET 30'
   'NET 30' → 'NET 30'
   'NET 45' → 'NET 45'
   'NET 60' → 'NET 60'
   'Due On Receipt' → 'Due On Receipt'
✅ Transformed 2 BANs
   - Mapped 12 fields per BAN
   - Set OwnerId to 005Ea00000ZOGFZIA5
   - Set busUnit__c to EVS
   - Set ES_Legacy_ID__c for tracking

⚠️  Skipped 184 BANs (invalid Payment_Terms__c)
   - GLCMI028899A: Invalid Payment_Terms__c value: 'NET90' - no BBF equivalent
   - A91910159838-R: Invalid Payment_Terms__c value: 'NET30,NET45,NET60' - no BBF equivalent
   - E91910237774-R: Invalid Payment_Terms__c value: 'NET30,NET45,NET60' - no BBF equivalent
   - E91910237816-R: Invalid Payment_Terms__c value: 'NET30,NET45,NET60' - no BBF equivalent
   - LYNX051602: Invalid Payment_Terms__c value: 'NET90' - no BBF equivalent


In [16]:
# === INSERT TO BBF ===

print("\n" + "=" * 80)
print("INSERTING BANs TO BBF")
print("=" * 80)

if len(bbf_bans) == 0:
    print("⚠️  No BANs to insert")
    successful_inserts = []
    failed_inserts = []
else:
    print(f"Inserting {len(bbf_bans)} BANs using bulk API...")
    print("(Bulk API automatically batches in 200-record chunks)\n")

    try:
        results = bbf_sf.bulk.BAN__c.insert(bbf_bans)

        successful_inserts = []
        failed_inserts = []

        for i, result in enumerate(results):
            if result["success"]:
                successful_inserts.append(
                    {
                        "es_id": bbf_bans[i]["ES_Legacy_ID__c"],
                        "bbf_id": result["id"],
                        "name": bbf_bans[i].get("Name", "N/A"),
                        "bbf_account_id": bbf_bans[i]["Account__c"],
                    }
                )
            else:
                failed_inserts.append(
                    {
                        "es_id": bbf_bans[i]["ES_Legacy_ID__c"],
                        "name": bbf_bans[i].get("Name", "N/A"),
                        "errors": result["errors"],
                        "bbf_account_id": bbf_bans[i]["Account__c"],
                    }
                )

        print(f"✅ Successfully inserted: {len(successful_inserts)} BANs")
        print(f"❌ Failed to insert: {len(failed_inserts)} BANs")

        if len(failed_inserts) > 0:
            print(f"\nFailed BANs (first 5):")
            for item in failed_inserts[:5]:
                print(f"  - {item['name']} (ES ID: {item['es_id']})")
                print(f"    Errors: {item['errors']}")

        if len(successful_inserts) > 0:
            print(f"\nSample successful insert:")
            sample = successful_inserts[0]
            print(f"  ES BAN ID:  {sample['es_id']}")
            print(f"  BBF BAN ID: {sample['bbf_id']}")
            print(f"  Name:       {sample['name']}")
            print(f"  BBF Account ID: {sample['bbf_account_id']}")

    except Exception as e:
        print(f"❌ Error during bulk insert: {e}")
        successful_inserts = []
        failed_inserts = []


INSERTING BANs TO BBF
Inserting 2 BANs using bulk API...
(Bulk API automatically batches in 200-record chunks)

✅ Successfully inserted: 2 BANs
❌ Failed to insert: 0 BANs

Sample successful insert:
  ES BAN ID:  aA33g000000CaeSCAS
  BBF BAN ID: a3BEa000009h3zyMAA
  Name:       EV-A91910231207-R
  BBF Account ID: 001Ea00001LEeQPIA1


In [17]:
# === UPDATE ES WITH BBF IDS ===

print("\n" + "=" * 80)
print("UPDATING ES WITH BBF IDS")
print("=" * 80)

if len(successful_inserts) == 0:
    print("⚠️  No BANs to update in ES")
    es_update_results = []
else:
    # Build update records for ES - set BBF_New_Id__c only
    es_updates = [
        {"Id": item["es_id"], "BBF_New_Id__c": item["bbf_id"]}
        for item in successful_inserts
    ]

    print(f"Updating {len(es_updates)} Billing_Invoice__c records in ES...")
    print("   - Setting BBF_New_Id__c = BBF BAN ID")

    try:
        es_update_results = es_sf.bulk.Billing_Invoice__c.update(es_updates)

        success_count = sum(1 for r in es_update_results if r["success"])
        error_count = sum(1 for r in es_update_results if not r["success"])

        print(
            f"\n✅ Successfully updated: {success_count} Billing_Invoice__c records in ES"
        )
        print(f"❌ Failed to update: {error_count} Billing_Invoice__c records in ES")

        if error_count > 0:
            print("\nFirst 10 update failures:")
            fail_count = 0
            for i, r in enumerate(es_update_results):
                if not r["success"] and fail_count < 10:
                    print(f"  - {es_updates[i]['Id']}: {r['errors']}")
                    fail_count += 1

    except Exception as e:
        print(f"❌ Error during ES update: {e}")
        es_update_results = []


UPDATING ES WITH BBF IDS
Updating 2 Billing_Invoice__c records in ES...
   - Setting BBF_New_Id__c = BBF BAN ID

✅ Successfully updated: 2 Billing_Invoice__c records in ES
❌ Failed to update: 0 Billing_Invoice__c records in ES


In [18]:
# === CREATE EXCEL OUTPUT ===

print("\n" + "=" * 80)
print("CREATING EXCEL OUTPUT")
print("=" * 80)

wb = Workbook()
ws1 = wb.active
ws1.title = "Migration Results"

# Styles
header_font = Font(bold=True, size=12, color="FFFFFF")
header_fill = PatternFill(start_color="4472C4", end_color="4472C4", fill_type="solid")
header_alignment = Alignment(horizontal="center", vertical="center")
thin_border = Border(
    left=Side(style="thin"),
    right=Side(style="thin"),
    top=Side(style="thin"),
    bottom=Side(style="thin"),
)

# Status colors
status_colors = {
    "Success": "C6EFCE",  # Green
    "Failed": "FFC7CE",  # Red
    "Skipped": "FFEB9C",  # Yellow
}

# --- SHEET 1: Migration Results ---
headers1 = [
    "ES Billing_Invoice ID",
    "BBF BAN ID",
    "BAN Name",
    "BBF Account ID",
    "Status",
    "Error",
]
ws1.append(headers1)

for col, header in enumerate(headers1, 1):
    cell = ws1.cell(row=1, column=col)
    cell.font = header_font
    cell.fill = header_fill
    cell.alignment = header_alignment
    cell.border = thin_border

# Combine all results
all_results = []
for item in successful_inserts:
    all_results.append(
        {
            "ES_ID": item["es_id"],
            "BBF_ID": item["bbf_id"],
            "Name": item["name"],
            "BBF_Account_ID": item["bbf_account_id"],
            "Status": "Success",
            "Error": "",
        }
    )
for item in failed_inserts:
    all_results.append(
        {
            "ES_ID": item["es_id"],
            "BBF_ID": "",
            "Name": item["name"],
            "BBF_Account_ID": item["bbf_account_id"],
            "Status": "Failed",
            "Error": str(item["errors"]),
        }
    )
for item in skipped_no_bbf_account:
    all_results.append(
        {
            "ES_ID": item["es_id"],
            "BBF_ID": "",
            "Name": item["name"],
            "BBF_Account_ID": "",
            "Status": "Skipped",
            "Error": "No BBF Account ID found",
        }
    )
for item in skipped_invalid_payment_terms:
    all_results.append(
        {
            "ES_ID": item["es_id"],
            "BBF_ID": "",
            "Name": item["name"],
            "BBF_Account_ID": item.get("bbf_account_id", ""),
            "Status": "Failed",
            "Error": item["reason"],
        }
    )

for row_idx, r in enumerate(all_results, 2):
    ws1.append(
        [
            r["ES_ID"],
            r["BBF_ID"],
            r["Name"],
            r["BBF_Account_ID"],
            r["Status"],
            r["Error"],
        ]
    )
    fill_color = status_colors.get(r["Status"], "FFFFFF")
    for col in range(1, len(headers1) + 1):
        cell = ws1.cell(row=row_idx, column=col)
        cell.fill = PatternFill("solid", fgColor=fill_color)
        cell.border = thin_border

for col in ws1.columns:
    max_length = max(len(str(cell.value)) for cell in col)
    ws1.column_dimensions[col[0].column_letter].width = min(max_length + 2, 60)

ws1.freeze_panes = "A2"

# --- SHEET 2: Summary ---
ws2 = wb.create_sheet("Summary")
ws2.append(["ES → BBF BAN Migration Summary"])
ws2["A1"].font = Font(bold=True, size=14)
ws2.append([])
ws2.append(["Run Type:", "TEST MODE" if TEST_MODE else "FULL MIGRATION"])
ws2.append(["Timestamp:", datetime.now().strftime("%Y-%m-%d %H:%M:%S")])
ws2.append(["Owner ID:", OWNER_ID])
ws2.append(["Business Unit:", DEFAULT_BUS_UNIT])
ws2.append([])
ws2.append(["Metric", "Count"])
ws2["A8"].font = Font(bold=True)
ws2["B8"].font = Font(bold=True)
ws2.append(["Total BANs Processed", len(all_results)])
ws2.append(["Successful Inserts", len(successful_inserts)])
ws2.append(["Failed Inserts", len(failed_inserts)])
ws2.append(["Failed - Invalid Payment Terms", len(skipped_invalid_payment_terms)])
ws2.append(["Skipped (No BBF Account)", len(skipped_no_bbf_account)])
ws2.append(
    [
        "Success Rate",
        (
            f"{len(successful_inserts)/len(all_results)*100:.1f}%"
            if len(all_results) > 0
            else "0%"
        ),
    ]
)

# --- SHEET 3: ID Mapping ---
ws3 = wb.create_sheet("ID Mapping")
headers3 = ["ES Billing_Invoice ID", "BBF BAN ID", "BBF Account ID", "BAN Name"]
ws3.append(headers3)

for col, header in enumerate(headers3, 1):
    cell = ws3.cell(row=1, column=col)
    cell.font = header_font
    cell.fill = header_fill
    cell.alignment = header_alignment

for item in successful_inserts:
    ws3.append([item["es_id"], item["bbf_id"], item["bbf_account_id"], item["name"]])

for col in ws3.columns:
    max_length = max(len(str(cell.value)) for cell in col)
    ws3.column_dimensions[col[0].column_letter].width = min(max_length + 2, 50)

ws3.freeze_panes = "A2"

# --- SHEET 4: Failed Inserts ---
ws4 = wb.create_sheet("Failed Inserts")
headers4 = ["ES Billing_Invoice ID", "BAN Name", "BBF Account ID", "Error Details"]
ws4.append(headers4)

for col, header in enumerate(headers4, 1):
    cell = ws4.cell(row=1, column=col)
    cell.font = Font(bold=True, size=12, color="FFFFFF")
    cell.fill = PatternFill(start_color="FF4444", end_color="FF4444", fill_type="solid")
    cell.alignment = header_alignment

for item in failed_inserts:
    ws4.append(
        [
            item["es_id"],
            item["name"],
            item["bbf_account_id"],
            str(item["errors"]),
        ]
    )

# Also add invalid payment terms to failed inserts
for item in skipped_invalid_payment_terms:
    ws4.append(
        [
            item["es_id"],
            item["name"],
            item.get("bbf_account_id", ""),
            item["reason"],
        ]
    )

for col in ws4.columns:
    max_length = max(len(str(cell.value)) for cell in col) if list(col) else 10
    ws4.column_dimensions[col[0].column_letter].width = min(max_length + 2, 70)

ws4.freeze_panes = "A2"

# Save workbook
wb.save(output_file)
print(f"\n✅ Excel output saved to: {output_file}")
print(f"   📊 Sheet 1: Migration Results ({len(all_results)} BANs, color-coded)")
print(f"   📈 Sheet 2: Summary (metrics and stats)")
print(f"   🔗 Sheet 3: ID Mapping ({len(successful_inserts)} successful mappings)")
print(
    f"   ⚠️  Sheet 4: Failed Inserts ({len(failed_inserts) + len(skipped_invalid_payment_terms)} failures)"
)


CREATING EXCEL OUTPUT

✅ Excel output saved to: es_bbf_ban_migration_20260123_120158.xlsx
   📊 Sheet 1: Migration Results (186 BANs, color-coded)
   📈 Sheet 2: Summary (metrics and stats)
   🔗 Sheet 3: ID Mapping (2 successful mappings)
   ⚠️  Sheet 4: Failed Inserts (184 failures)


In [19]:
# === FINAL SUMMARY ===

print("\n" + "=" * 80)
print("MIGRATION COMPLETE")
print("=" * 80)
print(f"ES Billing_Invoice__c queried: {len(es_bans)}")
print(f"BBF BAN__c inserted: {len(successful_inserts)}")
print(
    f"Success rate: {len(successful_inserts)/len(es_bans)*100:.1f}%"
    if len(es_bans) > 0
    else "N/A - No BANs processed"
)
print(f"\nExcel output: {output_file}")

if TEST_MODE:
    print(f"\n🔄 TEST MODE complete. Only migrated {TEST_LIMIT} BANs.")
    print("   To migrate ALL BANs, set TEST_MODE = False in Cell 2 and re-run.")
else:
    print("\n✅ FULL MIGRATION complete!")
    print("   BAN migration finished.")
    print("   Next: Migrate BAN_Contact__c, BAN_Team__c, Opportunities, etc.")

if len(failed_inserts) > 0:
    print(f"\n⚠️  {len(failed_inserts)} BANs failed to insert")
    print("   Check 'Failed Inserts' sheet in Excel for details")


MIGRATION COMPLETE
ES Billing_Invoice__c queried: 186
BBF BAN__c inserted: 2
Success rate: 1.1%

Excel output: es_bbf_ban_migration_20260123_120158.xlsx

✅ FULL MIGRATION complete!
   BAN migration finished.
   Next: Migrate BAN_Contact__c, BAN_Team__c, Opportunities, etc.


---
## Next Steps: Additional Object Migration

After BAN migration is complete, use the **ID Mapping sheet** from this Excel output to migrate related objects:

1. **BAN_Contact__c** (needs BAN + Contact IDs) - Junction object linking BANs to Contacts
2. **BAN_Team__c** (needs BAN ID) - Teams assigned to BANs
3. **Service__c** (needs BAN ID) - Active services linked to billing accounts

## Field Mapping Reference

### ES Billing_Invoice__c → BBF BAN__c Field Mapping

| ES Field | BBF Field | Notes |
|----------|-----------|-------|
| Id | ES_Legacy_ID__c | Tracking |
| Account__c | Account__c | Via BBF_New_Id__c lookup |
| Name | Name | BAN Name/Number |
| Billing_Address_1__c + Billing_Address_2__c | Billing_Street__c | Combined, max 100 chars |
| Billing_City__c | Billing_City__c | Direct map |
| Billing_State__c | Billing_State__c | Picklist - verify values match |
| Billing_ZIP__c | Billing_PostalCode__c | Direct map |
| Account_Name__c | Billing_Company_Name__c | Direct map |
| Description__c | BAN_Description__c | Direct map |
| Billing_Notes__c | General_Description__c | Direct map |
| Payment_Terms__c | Payment_Terms__c | Translated: NET30→NET 30, NET45→NET 45, NET60→NET 60 |
| (default) | busUnit__c | Required - set to "EverStream" |

## Cleanup Apex (if needed)

### Delete Migrated BANs from BBF
```apex
List<BAN__c> bans = [SELECT Id, Name FROM BAN__c WHERE ES_Legacy_ID__c != null];
System.debug('Found ' + bans.size() + ' migrated BANs');
delete bans;
```

### Remove BBF_New_Id__c from ES (to re-run migration)
```apex
List<Billing_Invoice__c> bis = [SELECT Id, BBF_New_Id__c 
                                 FROM Billing_Invoice__c 
                                 WHERE BBF_New_Id__c != NULL];
System.debug('Found ' + bis.size() + ' records to reset');
for (Billing_Invoice__c bi : bis) {
    bi.BBF_New_Id__c = NULL;
}
update bis;
```

In [20]:
# Install required packages (run if needed)
# !pip install simple-salesforce pandas openpyxl